# Sixt Data Science Lab - Test Task for Data Scientist Job Candidates

## Introduction

In this test task you will have an opportunity to demonstrate your skills of a Data Scientist from various angles - processing data, analyzing and vizalizing it, finding insights, applying predictive techniques and explaining your reasoning about it.

The task is based around a bike sharing dataset openly available at UCI Machine Learning Repository [1].

Please go through the steps below, build up the necessary code and comment on your choices.

## Part 1 - Data Loading and Environment Preparation

**Tasks:**
1. Prepare a Python 3 virtual environment (with virtualenv command). requirements.txt output of pip freeze command should be included as part of your submission.
2. Load the data from UCI Repository and put it into the same folder with the notebook. The link to it is https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset . Here is an available mirror in case the above website is down: https://data.world/uci/bike-sharing-dataset
3. We split the data into two parts. One dataset containing the last 30 days and one dataset with the rest.

## <span style="color:#436EEE"> Setup Environment

### Add utils

In [ ]:
import os, sys
os.getcwd()

# subfolders
print(os.listdir("data"))
print(os.listdir("output"))

### Confirm virtual environment

In [ ]:
print(sys.prefix)
print(sys.executable)

### Libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns
import sweetviz as sv
from scipy.stats import scoreatpercentile
from statsmodels.graphics.gofplots import qqplot
import time
import math

from sklearn import preprocessing, metrics, linear_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, StratifiedKFold,  cross_val_score, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle

import warnings
warnings.filterwarnings('ignore')

### Setup info

In [ ]:
# Config display options
pd.options.display.max_colwidth = 10000
pd.options.display.float_format = '{:.2f}'.format

# Display all outputs in Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# I want pandas to show all columns and up to * rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

# Environment for images
# This sets reasonable defaults for font size for
# a figure that will go in a notebook
sns.set_context("notebook")

# Set the font to be serif, rather than sans
sns.set(font='serif')

# Make the background white, and specify the
# specific font family
sns.set_style("whitegrid")

### <span style="color:#436EEE"> Load Data

In [ ]:
# read raw training data
df_all = pd.read_csv('data/day.csv')
df_hour_all = pd.read_csv('data/hour.csv')

# split dataset
df_last30 = df_all.tail(30) # use to test data in unseen data
df = df_all.iloc[:-30, :] # use to train data

df.head()
df.shape

## Part 2 - Data Processing and Analysis

**Tasks:**
1. Perform all needed steps to load and clean the data. Please comment the major steps of your code. 
2. Visualise rentals of bikes per day. 
3. Assume that each bike has exactly maximum 12 rentals per day.
    * Find the maximum number of bicycles `nmax` that was needed in any one day. <span style="color:#436EEE"> answer here
    * Find the 95%-percentile of bicycles `n95` that was needed in any one day. <span style="color:#436EEE"> answer here
5. Visualize the distribution of the covered days depending on the number of available bicycles (e.g. `nmax` bicycles would cover 100% of days, `n95` covers 95%, etc.)


### <span style="color:#436EEE"> Dataset characteristics

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv
	
	- instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

## <span style="color:#436EEE"> Storytelling
### <span style="color:#436EEE"> EDA - Understanding Data

In [ ]:
df_all.info()
df_all.describe()

#### Report Overview

In [ ]:
# Report about total dataset, with target feature
eda_report = sv.analyze([df_all,'Bike Rentals'], 'cnt')
eda_report.show_html('output/analyze_dataset.html')
eda_report.show_notebook(layout='widescreen', w=1500, h=700, scale=0.7)

**Comments / reasoning:**

- I observe a strong positive correlation between registered and total rents
- Casual users have a moderate correlation with total rents
- We have less total rents during Spring months (more rentals during Summer/Autumn) and we observe much more rentals during holidays, since it is really conducive to ride bike in that season and moment of the week. Therefore June, July, August and September has relatively higher demand for bicycle.
- Users prefer to rent bikes when windspeed is < 0.2 (low windspeed), however there's an interesting pike when windspeed is around 0.45
- Humidity only has a negotive influence for bike rentals when is < 0.8 (people not comfortable with sweating too much...)
- When the temperature is higher we observe higher rentals 
- There's a great increase in bike rentals from 2011 to 2012

In [ ]:
# Report comparison about split datasets, with target feature
eda_report_comparison = sv.compare([df_all, 'training data'], [df_last30, 'last 30 days'], 'cnt')
eda_report_comparison.show_html('output/analyze_dataset_comparison.html')
eda_report_comparison.show_notebook(layout='widescreen', w=1500, h=700, scale=0.7)

**Comments / reasoning:**

- Different behaviour bike rentals during weekdays, last 30 days dataset has more rentals on Wednesdays and very low on Mondays (if  0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday, 5 = Saturday, 6 = Sunday)
- From last 30 days dataset I observe that windspeed impacted more rentals when between 0.2 and 0.3.

### <span style="color:#436EEE"> EDA - Cleaning Data

- <span style="color:#436EEE"> Remove one of the temperature variables, because they are highly correlated

In [ ]:
# rename columns
df_all.rename(columns={'instant':'id','dteday':'datetime','yr':'year','mnth':'month','weathersit':'weather_condition',
                       'temp':'temperature', 'atemp':'feel_temperature', 'hum':'humidity','cnt':'total_count'},inplace=True)
df_all.head()
df_all.dtypes

In [ ]:
df_all['datetime']=pd.to_datetime(df_all.datetime)
df_all['season']=df_all.season.astype('category')
df_all['year']=df_all.year.astype('category')
df_all['month']=df_all.month.astype('category')
df_all['holiday']=df_all.holiday.astype('category')
df_all['weekday']=df_all.weekday.astype('category')
df_all['workingday']=df_all.workingday.astype('category')
df_all['weather_condition']=df_all.weather_condition.astype('category')

df_all.dtypes

- **Missings**

In [ ]:
print('df_all shape : ' + str(df_all.shape))

# Split dataframe by numerical and categorical columns
num_df = df_all.select_dtypes(include = ['int64', 'float64'])
cat_df = df_all.select_dtypes(include = ['object', 'bool'])

# Get list of columns with missing values
missing_num = num_df.isnull().sum()
columns_with_missing_num = missing_num[missing_num > 0]
print("**These are the NUMERIC columns with missing values:**\n{} \n"\
      .format(columns_with_missing_num))

# Get list of columns with missing values
missing_cat = cat_df.isnull().sum()
columns_with_missing_cat = (missing_cat[(missing_cat > 0) & (missing_cat < len(df_all))])
print("**These are the CATEGORICAL columns with missing values:**\n{} \n"\
      .format(columns_with_missing_cat))

columns_with_all_missing_num = missing_num[missing_num == len(df_all)]
columns_with_all_missing_num = list(columns_with_all_missing_num.index)
print("**These are the NUMERICAL columns with ALL missing values:**\n{} \n"\
      .format(columns_with_all_missing_num))

columns_with_all_missing_cat = missing_cat[missing_cat == len(df_all)]
columns_with_all_missing_cat = list(columns_with_all_missing_cat.index)
print("**These are the CATEGORICAL columns with ALL missing values:**\n{}"\
      .format(columns_with_all_missing_cat))

df_all.drop(columns_with_all_missing_num, axis = 1, inplace = True)
df_all.drop(columns_with_all_missing_cat, axis = 1, inplace = True)

- **Handling High Cardinality in Categorical columns**

In [ ]:
# Considering cardinality_threshold
cardinality_threshold = 10

# Get list of columns with their cardinality - don't want to consider numeric columns
categorical_columns = list(df_all.select_dtypes(exclude=[np.number]).columns)
cardinality = df_all[categorical_columns].apply(pd.Series.nunique)
columns_too_high_cardinality = list(cardinality[cardinality > cardinality_threshold].index)
print("There are {} columns with high cardinality. Threshold: {} categories."\
      .format(len(columns_too_high_cardinality), cardinality_threshold))
columns_too_high_cardinality

- **Remove ID & unnecessary columns**

In [ ]:
ID_variables = ['id']

df_all.drop(ID_variables, axis = 1, inplace = True)

In [ ]:
# casual & registered – These variables cannot be predicted
no_value_variables = ['casual', 'registered']

df_all.drop(no_value_variables, axis = 1, inplace = True)

- **Remove constant columns**

In [ ]:
# get list of columns with constant value
columns_constant = list(df_all.columns[df_all.nunique() <= 1])
print("There are {} columns with constant values".format(len(columns_constant)))
columns_constant

df_all.drop(columns_constant, axis = 1, inplace = True)

- **Remove perfect correlated**

In [ ]:
corr_matrix = df_all.select_dtypes(exclude=[np.object]).corr().abs()

# select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# find features with correlation > 0.95
columns_perfect_correlation = [column for column in upper.columns if any(upper[column] > 0.95)]
print("There are {} columns that are perfectly correlated with other columns: {} "\
          .format(len(columns_perfect_correlation), columns_perfect_correlation))

columns_perfect_correlation

In [ ]:
# I prefer to drop 'temp' the column atemp is more appropriate for modelling purposes, from human perspective
df_all.drop('temperature', axis=1, inplace=True)

- **Outliers**

In [ ]:
fig,ax = plt.subplots(figsize = (5, 3) )

# boxplot for total_count outliers
sns.boxplot(data = df_all[['total_count']])
ax.set_title('total_count outliers')
plt.show()

In [ ]:
# plot box plot of categorical variables

plt.figure(figsize=(20, 12))
plt.subplot(3,3,1)
sns.boxplot(x = 'season', y = 'total_count', data = df_all)
plt.subplot(3,3,2)
sns.boxplot(x = 'year', y = 'total_count', data = df_all)
plt.subplot(3,3,3)
sns.boxplot(x = 'month', y = 'total_count', data = df_all)
plt.subplot(3,3,4)
sns.boxplot(x = 'holiday', y = 'total_count', data = df_all)
plt.subplot(3,3,5)
sns.boxplot(x = 'weekday', y = 'total_count', data = df_all)
plt.subplot(3,3,6)
sns.boxplot(x = 'workingday', y = 'total_count', data = df_all)
plt.subplot(3,3,7)
sns.boxplot(x = 'weather_condition', y = 'total_count', data = df_all)
plt.show()

In [ ]:
# plot box plot of continuous variables

plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)
plt.boxplot(df_all["feel_temperature"])
plt.subplot(2,3,3)
plt.boxplot(df_all["humidity"])
plt.subplot(2,3,4)
plt.boxplot(df_all["windspeed"])
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (7, 5))

# zoom in for outliers regarding windspeed & humidity features
sns.boxplot(data = df_all[['windspeed','humidity']])
ax.set_title('Windspeed_Humidity outliers')
plt.show()

- **Replace and impute the outliers**

In [ ]:
from fancyimpute import KNN

# create dataframe for outliers
outliers = pd.DataFrame(df_all, columns=['windspeed','humidity'])

# replace outliers by n/a
columns = ['windspeed','humidity']
for i in columns:
    q75, q25 = np.percentile(outliers.loc[:,i],[75,25]) # Split data in 2 diff quantiles
    iqr = q75 - q25 # inter quantile range
    min = q25 - (iqr*1.5)
    max = q75 + (iqr*1.5) 
    outliers.loc[outliers.loc[:,i] < min, :i] = np.nan
    outliers.loc[outliers.loc[:,i] > max, :i] = np.nan

# impute outliers by using the average
outliers['windspeed'] = outliers['windspeed'].fillna(outliers['windspeed'].mean())
outliers['humidity'] = outliers['humidity'].fillna(outliers['humidity'].mean())

- **Replace the original dataset by imputated dataset**

In [ ]:
#Replacing the imputated windspeed
df_all['windspeed'] = df_all['windspeed'].replace(outliers['windspeed'])

#Replacing the imputated humidity
df_all['humidity'] = df_all['humidity'].replace(outliers['humidity'])
df_all.head(5)

- **Normal Probability Plot**

In [ ]:
fig = plt.figure(figsize=(15,8))
stats.probplot(df_all.total_count.tolist(), dist='norm',plot=plt)
plt.show()

- **Correlation Matrix**

In [ ]:
# using Pearson Correlation
df = df_all[df_all.columns]

plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
plt.show()

**Comments / reasoning:**

- Regarding first total_count box plot, I observe no outliers in total bike rentals in this dataset
- Regarding windspeed & humidity box plot analysis, I only observe outliers in windspeed and humidity features in this dataset
- Regarding probability plot, there are few target variable data points that deviates from normality 
- Regarding the correlation matrix, I observe a significant positive correlation between season_fall and feel_temperature and also the target variable with feel_temperature

### <span style="color:#436EEE"> Feature Engineering

In [ ]:
season_type = pd.get_dummies(df_all['season'], drop_first = True)
season_type.rename(columns={2:"season_summer", 3:"season_fall", 4:"season_winter"},inplace=True)
season_type.head()

weather_type = pd.get_dummies(df_all['weather_condition'], drop_first = True)
weather_type.rename(columns={2:"weather_mist_cloud", 3:"weather_light_snow_rain"},inplace=True)
weather_type.head()

In [ ]:
# concatenate new dummy variables to df_all
df_all = pd.concat([df_all, season_type, weather_type], axis = 1)

# drop previous columns season & weathersit
df_all.drop(columns=["season", "weather_condition"],axis=1, inplace =True)
df_all.info()

### <span style="color:#436EEE"> Questions and Answers
2. Visualise rentals of bikes per day. 
3. Assume that each bike has exactly maximum 12 rentals per day.
    * Find the maximum number of bicycles `nmax` that was needed in any one day.
    * Find the 95%-percentile of bicycles `n95` that was needed in any one day. 
4. Visualize the distribution of the covered days depending on the number of available bicycles (e.g. `nmax` bicycles would cover 100% of days, `n95` covers 95%, etc.)

In [ ]:
# calculate rentals of bikes per day of the week
total_rents_by_day = df_all[['datetime', 'total_count']]
#total_rents_by_day

# visualize data
plt.figure(figsize = (30, 15))

fig = px.line(total_rents_by_day, x = 'datetime', y = 'total_count', title = 'Total Rentals per Day')
fig.show()

In [ ]:
# Max Number of Bikes = Total requested riders / Max no of rides per bike
# Max Number of Bikes = total_count / 12

df_all["total_count_max12"] = df_all["total_count"]/12
#df_all.head()

# calculate the maximum number of bicycles nmax that was needed in any one day
nmax = pd.Series(df_all["total_count_max12"])
print("The maximum number of bicycles nmax that was needed in any one day is", round(nmax.quantile(1, 'nearest'), 1), "!")

# calculate the 95%-percentile of bicycles n95 that was needed in any one day
n95 = pd.Series(df_all["total_count_max12"])
print("The 95%-percentile of bicycles n95 that was needed in any one day is", round(nmax.quantile(0.95, 'nearest'), 1), "!")

In [ ]:
a = list(range(1,101))
b = [scoreatpercentile(df_all["total_count"],i) for i in a]

df2 = pd.DataFrame({'percentile': a, 'total_count': b}, columns=['percentile', 'total_count'])
fig = px.line(df2, x = 'percentile', y = 'total_count', title = 'Distribution of the Covered Days Depending on the Number of Available Bicycles')
fig

## Part 3 - Building prediction models

**Tasks:**
1. Define a test metric for predicting the daily demand for bike sharing, which you would like to use to measure the accuracy of the constructed models, and explain your choice.
2. Build a demand prediction model with Random Forest, preferably making use of following python libraries: scikit-learn. 
3. Report the value of the chosen test metric on the provided data. 

### <span style="color:#436EEE"> Define Test Metric

Bike sharing demand prediction refers to the process of forecasting the number of bicycles that will be rented within a specific time period, aiding in resource allocation and system optimization.
For predicting the daily demand for bike sharing, which is a regression model, since the target variable is a quantity (over time) and consequent model evaluation to access the performance of this forecasting model, I'll use various metrics to evaluate the performance:
- mean absolute error (MAE)
- root mean squared error (RMSE)
- coefficient of determination (R-squared).

MAE and RMSE measure the average magnitude of the errors between the predicted and actual values. \
R-squared measures the proportion of variance in the target variable, explained by the input variables.

### <span style="color:#436EEE"> Train a Regression Model

Next step is to train a Regression Model (in this case Random Forest), which will use the potentially predictive features we have identified to forecast the “total_count” label

In [ ]:
# define training dataset
df = df_all.iloc[:-30, :]

df.columns
df.dtypes
df.head(2)

- **Set target variable**

In [ ]:
# dump not needed columns
training_data = df.drop(['datetime', 'total_count_max12'], axis=1)

# move total_count as last column
training_data = training_data[ [ col for col in training_data.columns if col != 'total_count' ] + ['total_count']]
training_data.head()

- **Split into train and test data**

In [ ]:
# split the dataset into the train and test data
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:,0:-1], training_data.iloc[:,-1], test_size = 0.2, random_state = 0)

print('x train :', X_train.shape,'\t\tx test :', X_test.shape)
print('y train :', y_train.shape,'\t\ty test :', y_test.shape)

- **Split the features into categorical and numerical features**

In [ ]:
# create a new dataset for train attributes
train_attributes = X_train[X_train.columns]

# create a new dataset for test attributes
test_attributes = X_test[X_test.columns]

# split dataframe by numerical and categorical columns
num_cols = X_train.select_dtypes(include = ['uint8', 'int64', 'float64']).columns
cat_cols = X_train.select_dtypes(include = ['object', 'bool', 'category']).columns

print("There are {} numeric columns and {} categorical columns".format(len(num_cols), len(cat_cols)))

- **Decoding the training attributes**

In [ ]:
# get dummy variables to encode the categorical features to numeric
train_encoded_attributes = pd.get_dummies(train_attributes, columns = cat_cols)

print('Shape of transfomed dataframe:', train_encoded_attributes.shape)
train_encoded_attributes.head(2)

- **Decoding the training attributes**

In [ ]:
# training dataset for modelling
X_train = train_encoded_attributes
y_train = y_train #.total_count.values

In [ ]:
# training the model
X_train = train_encoded_attributes
model = RandomForestRegressor(random_state = 0, n_estimators = 200)

In [ ]:
# fit the trained model
model.fit(X_train, y_train)

- **Cross validation prediction**

Cross-validation is used to estimate the performance of machine learning models, more specificaly, it is used to protect against overfitting in a predictive model, particularly in a case where the amount of data may be limited.

In [ ]:
predict = cross_val_predict(model, X_train, y_train, cv=3)

In [ ]:
# cross validation prediction plot
fig,ax = plt.subplots(figsize=(15,8))
ax.scatter(y_train, y_train-predict)
ax.axhline(lw=2,color='black')
ax.set_title('Cross validation prediction plot')
ax.set_xlabel('Observed')
ax.set_ylabel('Residual')
#plt.show()

# calculate equation for trendline
z = np.polyfit(y_train, y_train-predict, 1)
p = np.poly1d(z)

# add trendline to plot
plt.plot(y_train, p(y_train), color="lightgreen", linewidth=3, linestyle="--")

In [ ]:
# R-squared scores
r2_scores = cross_val_score(model, X_train, y_train, cv=5)
print('R^2 scores :', np.average(r2_scores))

**Answers / comments / reasoning:**

- Observing cross validation prediction plot we see there is an apparent diagonal trend and the points where the predicted and actual values intersect generally follow the trend line. There is a good fitness of the model in this case, however it some data points present a higher variation. Normally if there's a variation, it represents the model’s residuals, which are the differences between the predicted label and the actual value of the validation label when the model applies the coefficients it learned during training to the validation data. By assessing these residuals from the validation data, we can estimate the level of error that can be expected when the model is used with new data for which the label is unknown.
- The R-squared or coefficient of determination is ~ 85.7% on average for 5-fold cross validation, it means that predictor is only able to predict 85.7% of the variance in the target variable which is contributed by independent variables.

- **Decoding the test attributes**

In [ ]:
# get dummy variables to encode the categorical features to numeric
test_encoded_attributes=pd.get_dummies(test_attributes,columns=cat_cols)

print('Shape of transformed dataframe :', test_encoded_attributes.shape)
test_encoded_attributes.head(2)

### <span style="color:#436EEE"> Model performance on test dataset

In [ ]:
# predict the model
X_test = test_encoded_attributes
y_pred = model.predict(X_test)

In [ ]:
# R-squared scores
r2_scores = cross_val_score(model, X_test, y_test, cv=5)
print('R^2 scores :', np.average(r2_scores))

### <span style="color:#436EEE"> Model Optimization

In [ ]:
# find best value for n_estimators
max = 0
index = -1
for i in range(10, 200):
    model = RandomForestRegressor(random_state = 0, n_estimators = i)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_score = metrics.r2_score(y_test, y_pred)
    if r2_score > max:
        index = i
        max = r2_score

In [ ]:
# confirm same dimension for the target variable
y_test.shape
y_pred.shape

In [ ]:
model_opt = RandomForestRegressor(random_state = 0, n_estimators = index)
model_opt.fit(X_train, y_train)
y_pred = model_opt.predict(X_test)

mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
me = metrics.max_error(y_test, y_pred)

print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('Max Error:', me)

In [ ]:
# R-squared scores
r2_scores = cross_val_score(model_opt, X_test, y_test, cv=5)
print('R^2 scores :', np.average(r2_scores))

**Answers / comments / reasoning:**

- After parameter tuning, the R-squared or coefficient of determination is the same (~80.3%), it means that this optimization did not enhance the results of the model, probably because we have limited and small dataset and also the difference in n_estimators was less than 3%. It would at least optimize the running time if dataset were bigger and less balanced.

In [ ]:
# save the optimized trained model as a pickle file
saved_model = pickle.dumps('model/model_opt')
  
# Load the pickled model
rf_model_pkl = pickle.loads('model/saved_model')
  
# Use the loaded pickled model to make predictions
#rf_model_pkl.predict(X_test)

- **Residual plot**

In [ ]:
# residual scatter plot
fig, ax = plt.subplots(figsize=(15,8))
residuals=y_test-y_pred
ax.scatter(y_test, residuals)
ax.axhline(lw=2, color='black')
ax.set_xlabel('Observed')
ax.set_ylabel('Residuals')
ax.set_title('Residual plot')
#plt.show()

# calculate equation for trendline
z = np.polyfit(y_test, residuals, 1)
p = np.poly1d(z)

# add trendline to plot
plt.plot(y_test, p(y_test), color="lightgreen", linewidth=3, linestyle="--")

### <span style="color:#436EEE"> Predicting Bike Rental count on Daily basis, in Out-of-Sample Data

Out-of-sample testing is used to evaluate the performance of a strategy on a separate set of data that was not used during the development and optimisation process. \
This helps to determine whether the strategy would be able to perform well on new, unseen data

In [ ]:
# define out-of-sample dataset
df_last30 = df_all.tail(30)
df_last30.head()

In [ ]:
# save date variable
times = df_last30['datetime']

# dump not needed columns
testing_data = df_last30.drop(['datetime', 'total_count_max12'], axis=1)

# move total_count as last column
testing_data = testing_data[ [ col for col in testing_data.columns if col != 'total_count' ] + ['total_count']]
testing_data.head()
print('Shape of OOS dataframe :', testing_data.shape)

In [ ]:
# create a new dataset for test attributes
testing_data_attributes = testing_data[testing_data.columns]

# split dataframe by numerical and categorical columns
num_cols = testing_data.select_dtypes(include = ['uint8', 'int64', 'float64']).columns
cat_cols = testing_data.select_dtypes(include = ['object', 'bool', 'category']).columns

print("There are {} numeric columns and {} categorical columns".format(len(num_cols), len(cat_cols)))

# get dummy variables to encode the categorical features to numeric
testing_data_encoded_attributes = pd.get_dummies(testing_data_attributes, columns=cat_cols)

# drop target variable
testing_data_encoded_attributes = testing_data_encoded_attributes.drop(['total_count'], axis = 1)

print('Shape of transformed dataframe :', testing_data_encoded_attributes.shape)
testing_data_encoded_attributes.head(2)

In [ ]:
# make predictions
y_pred_testing = model_opt.predict(testing_data_encoded_attributes)

In [ ]:
# submit final sample
Submission = pd.DataFrame({'datetime' : times, 'pred' : y_pred_testing})
Submission.set_index('datetime', inplace = True)
Submission.to_csv('output/sample_submission.csv')
Submission

### <span style="color:#436EEE"> Final Conclusions

## Part 4 - Reflection / comments

**Tasks:**
(Optional) Please share with us any free form reflection, comments or feedback you have in the context of this test task.

In summary, this notebook conducted a comprehensive analysis of daily bike rental data in two years time basis. Hands-on in data exploration, preprocessing, and feature engineering to prepare the data for modeling. The exploratory data analysis provided valuable insights into rental patterns based on different factors, such as weather, day of the week, seasonality, etc.
In conclusion, this notebook gave us great insights on bike rental trends and successfully predicted rental counts using the Random Forest model.

Future improvements:
1. Exploratory Data Analysis using hour dataset
2. More feature engineering using day/hour dataset features
3. Tuning other parameters inside RF
4. Apply other models to compare performances
5. Employing advanced feature selection/explainability techniques (eg. SHAP)

The analysis and insights presented here can provide valuable guidance for bike-sharing companies to optimize their services and meet the diverse preferences of their user base.

## Submission

Please submit this notebook with your developments in .ipynb and .html formats as well as your requirements.txt file.

## References

[1] Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.